In [1]:
# Bibliotecas
import pandas as pd
import numpy as np
import glob

In [2]:
# Carregando os dados
# Listar os arquivos CSV na pasta 'uploads'
arquivo_csv = glob.glob('uploads/processos_*.csv')

# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
for arquivo in arquivo_csv:
    # Extrair o ano do nome do arquivo
    ano = int(arquivo.split('_')[-1].split('.')[0])
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    df_ano['ano_arquivo'] = ano  # Adicionar coluna com o ano do arquivo
    dfs.append(df_ano)

df = pd.concat(dfs, ignore_index=True)

print(df.shape)

(3245632, 11)


In [3]:
# Limpeza e tratamento
# Verificar o nome correto das colunas (pode haver diferenças de acentuação ou espaços)
colunas = df.columns.tolist()

# Encontrar as colunas de data corretamente
coluna_serventia = [col for col in colunas if 'serventia' in col.lower()][0]
coluna_distribuicao = [col for col in colunas if 'data_distribuicao' in col.lower()][0]
coluna_baixa = [col for col in colunas if 'data_baixa' in col.lower()][0]
coluna_area_acao = [col for col in colunas if 'nome_area_acao' in col.lower()][0]
coluna_processo_id = [col for col in colunas if 'processo' in col.lower()][0]
coluna_comarca = [col for col in colunas if 'comarca' in col.lower()][0]

# Renomear colunas para garantir consistência
df = df.rename(columns={
coluna_distribuicao: 'data_distribuicao',
coluna_baixa: 'data_baixa',
coluna_area_acao: 'nome_area_acao',
coluna_processo_id: 'processo',
coluna_comarca: 'comarca',
coluna_serventia: 'serventia'
})

# Converter colunas de data para datetime com tratamento de erros
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

In [ ]:
# CRIAÇÃO DAS ESTATÍSTICAS MENSAIS:

# 1. Extrair ano de distribuição e baixa
df['ano_distribuicao'] = df['data_distribuicao'].dt.year
df['ano_baixa'] = df['data_baixa'].dt.year
df['mes_distribuicao'] = df['data_distribuicao'].dt.month
df['mes_baixa'] = df['data_baixa'].dt.month

# Chave de agrupamento para as estatísticas
grouping_keys = ['comarca', 'serventia', 'nome_area_acao']

# 2. Calcular Distribuídos por ano
distribuidos_df = df.dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos')
distribuidos_df = distribuidos_df.rename(columns={'ano_distribuicao': 'ano'})

# 2.1 Calcular Distribuídos por mês
distribuidos_mes_df = df.dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos_mes')
distribuidos_mes_df = distribuidos_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# 3. Calcular Baixados por ano
baixados_df = df.dropna(subset=['ano_baixa']).groupby(
    ['ano_baixa'] + grouping_keys
).size().reset_index(name='Baixados')
baixados_df = baixados_df.rename(columns={'ano_baixa': 'ano'})

# 3.1 Calcular Baixados por mês
baixados_mes_df = df.dropna(subset=['ano_baixa', 'mes_baixa']).groupby(
    ['ano_baixa', 'mes_baixa'] + grouping_keys
).size().reset_index(name='Baixados_mes')
baixados_mes_df = baixados_mes_df.rename(columns={'ano_baixa': 'ano', 'mes_baixa': 'mes'})

# 4. Calcular Pendentes por ano de distribuição
pendentes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes')
pendentes_df = pendentes_df.rename(columns={'ano_distribuicao': 'ano'})

# 4.1 Calcular Pendentes por mês de distribuição
pendentes_mes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes_mes')
pendentes_mes_df = pendentes_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# 5. Juntar tudo em uma única tabela (estatisticas)
estatisticas = pd.merge(distribuidos_df, baixados_df, on=['ano'] + grouping_keys, how='outer')
estatisticas = pd.merge(estatisticas, pendentes_df, on=['ano'] + grouping_keys, how='outer')

# 5.1 Juntar tudo em uma única tabela (estatisticas_mes)
merge_keys = ['ano', 'mes'] + grouping_keys
estatisticas_mes = pd.merge(distribuidos_mes_df, baixados_mes_df, on=merge_keys, how='outer')
estatisticas_mes = pd.merge(estatisticas_mes, pendentes_mes_df, on=merge_keys, how='outer')

# 6. Limpar e organizar o DataFrame final
estatisticas = estatisticas.fillna(0)
estatisticas[['Distribuídos', 'Baixados', 'Pendentes']] = estatisticas[['Distribuídos', 'Baixados', 'Pendentes']].astype(int)

# 6.1 Limpar e organizar o DataFrame final
estatisticas_mes = estatisticas_mes.fillna(0)
estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']] = estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']].astype(int)

# Remover linhas onde o ano não pôde ser determinado
estatisticas = estatisticas.dropna(subset=['ano'])
estatisticas['ano'] = estatisticas['ano'].astype(int)

# Remover linhas onde o mês não pôde ser determinado
estatisticas_mes = estatisticas_mes.dropna(subset=['ano', 'mes'])
estatisticas_mes[['ano', 'mes']] = estatisticas_mes[['ano', 'mes']].astype(int)

# 7. Calcular a Taxa de Congestionamento por ano
soma_pend_baix = estatisticas['Pendentes'] + estatisticas['Baixados']
estatisticas['Taxa de Congestionamento (%)'] = np.where(
    soma_pend_baix > 0,
    (estatisticas['Pendentes'] / soma_pend_baix) * 100,
    0
).round(2)

# 7.1 Calcular a Taxa de Congestionamento por mês
soma_pend_baix_mes = estatisticas_mes['Pendentes_mes'] + estatisticas_mes['Baixados_mes']
estatisticas_mes['Taxa de Congestionamento_mes (%)'] = np.where(
    soma_pend_baix_mes > 0,
    (estatisticas_mes['Pendentes_mes'] / soma_pend_baix_mes) * 100,
    0
).round(2)

# 8. Ordenar e reordenar colunas para melhor visualização
estatisticas = estatisticas.sort_values(by=['ano', 'comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True])

colunas_finais = [
    'ano', 'nome_area_acao', 'comarca', 'serventia', 
    'Distribuídos', 'Baixados', 'Pendentes', 'Taxa de Congestionamento (%)'
]
estatisticas = estatisticas[colunas_finais]

# 8.1 Ordenar e reordenar colunas para melhor visualização
estatisticas_mes = estatisticas_mes.sort_values(by=['ano', 'mes','comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True, True])

colunas_finais_mes = [
    'ano', 'mes', 'nome_area_acao', 'comarca', 'serventia', 
    'Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes', 'Taxa de Congestionamento_mes (%)'
]
estatisticas_mes = estatisticas_mes[colunas_finais_mes]

#print(estatisticas.head(10))


print(estatisticas_mes.head(10))


        ano  mes                    nome_area_acao    comarca  \
54007  2025    1                             civel  ABADIÂNIA   
54008  2025    1                          criminal  ABADIÂNIA   
54009  2025    1                familia - interior  ABADIÂNIA   
54010  2025    1             fazenda publica mista  ABADIÂNIA   
54011  2025    1  infancia e juventude infracional  ABADIÂNIA   
54012  2025    1            juizado especial civel  ABADIÂNIA   
54013  2025    1         juizado especial criminal  ABADIÂNIA   
54014  2025    1  juizado especial fazenda publica  ABADIÂNIA   
54015  2025    1                             civel    ACREÚNA   
54016  2025    1                familia - interior    ACREÚNA   

                                               serventia  Distribuídos_mes  \
54007                                      Vara Judicial                49   
54008                                      Vara Judicial                18   
54009                                      Vara Ju

In [4]:
# --- CRIAÇÃO DOS TABELÕES DE ESTATÍSTICAS (ANUAL, MENSAL, SEMANAL, DIÁRIA) ---
# 1. Extrair componentes de data
df['ano_distribuicao'] = df['data_distribuicao'].dt.year
df['mes_distribuicao'] = df['data_distribuicao'].dt.month
df['dia_distribuicao'] = df['data_distribuicao'].dt.day
df['ano_semana_dist'] = df['data_distribuicao'].dt.isocalendar().year
df['semana_dist'] = df['data_distribuicao'].dt.isocalendar().week

df['ano_baixa'] = df['data_baixa'].dt.year
df['mes_baixa'] = df['data_baixa'].dt.month
df['dia_baixa'] = df['data_baixa'].dt.day
df['ano_semana_baixa'] = df['data_baixa'].dt.isocalendar().year
df['semana_baixa'] = df['data_baixa'].dt.isocalendar().week

# Chave de agrupamento para as estatísticas
grouping_keys = ['comarca', 'serventia', 'nome_area_acao']

# --- 2. CÁLCULOS ANUAIS ---
print("Calculando estatísticas anuais...")
# 2.1 Distribuídos por ANO
distribuidos_df = df.dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos')
distribuidos_df = distribuidos_df.rename(columns={'ano_distribuicao': 'ano'})

# 2.2 Baixados por ANO
baixados_df = df.dropna(subset=['ano_baixa']).groupby(
    ['ano_baixa'] + grouping_keys
).size().reset_index(name='Baixados')
baixados_df = baixados_df.rename(columns={'ano_baixa': 'ano'})

# 2.3 Pendentes por ANO
pendentes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao']).groupby(
    ['ano_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes')
pendentes_df = pendentes_df.rename(columns={'ano_distribuicao': 'ano'})

# --- 3. CÁLCULOS MENSAIS ---
print("Calculando estatísticas mensais...")
# 3.1 Distribuídos por MÊS
distribuidos_mes_df = df.dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos_mes')
distribuidos_mes_df = distribuidos_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# 3.2 Baixados por MÊS
baixados_mes_df = df.dropna(subset=['ano_baixa', 'mes_baixa']).groupby(
    ['ano_baixa', 'mes_baixa'] + grouping_keys
).size().reset_index(name='Baixados_mes')
baixados_mes_df = baixados_mes_df.rename(columns={'ano_baixa': 'ano', 'mes_baixa': 'mes'})

# 3.3 Pendentes por MÊS
pendentes_mes_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao', 'mes_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes_mes')
pendentes_mes_df = pendentes_mes_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes'})

# --- 4. CÁLCULOS SEMANAIS (ISO) ---
print("Calculando estatísticas semanais...")
# 4.1 Distribuídos por SEMANA
distribuidos_semana_df = df.dropna(subset=['ano_semana_dist', 'semana_dist']).groupby(
    ['ano_semana_dist', 'semana_dist'] + grouping_keys
).size().reset_index(name='Distribuídos_semana')
distribuidos_semana_df = distribuidos_semana_df.rename(columns={'ano_semana_dist': 'ano_semana', 'semana_dist': 'semana'})

# 4.2 Baixados por SEMANA
baixados_semana_df = df.dropna(subset=['ano_semana_baixa', 'semana_baixa']).groupby(
    ['ano_semana_baixa', 'semana_baixa'] + grouping_keys
).size().reset_index(name='Baixados_semana')
baixados_semana_df = baixados_semana_df.rename(columns={'ano_semana_baixa': 'ano_semana', 'semana_baixa': 'semana'})

# 4.3 Pendentes por SEMANA
pendentes_semana_df = df[df['data_baixa'].isna()].dropna(subset=['ano_semana_dist', 'semana_dist']).groupby(
    ['ano_semana_dist', 'semana_dist'] + grouping_keys
).size().reset_index(name='Pendentes_semana')
pendentes_semana_df = pendentes_semana_df.rename(columns={'ano_semana_dist': 'ano_semana', 'semana_dist': 'semana'})

# --- 5. CÁLCULOS DIÁRIOS ---
print("Calculando estatísticas diárias...")
# 5.1 Distribuídos por DIA
distribuidos_dia_df = df.dropna(subset=['ano_distribuicao', 'mes_distribuicao', 'dia_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao', 'dia_distribuicao'] + grouping_keys
).size().reset_index(name='Distribuídos_dia')
distribuidos_dia_df = distribuidos_dia_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes', 'dia_distribuicao': 'dia'})

# 5.2 Baixados por DIA
baixados_dia_df = df.dropna(subset=['ano_baixa', 'mes_baixa', 'dia_baixa']).groupby(
    ['ano_baixa', 'mes_baixa', 'dia_baixa'] + grouping_keys
).size().reset_index(name='Baixados_dia')
baixados_dia_df = baixados_dia_df.rename(columns={'ano_baixa': 'ano', 'mes_baixa': 'mes', 'dia_baixa': 'dia'})

# 5.3 Pendentes por DIA
pendentes_dia_df = df[df['data_baixa'].isna()].dropna(subset=['ano_distribuicao', 'mes_distribuicao', 'dia_distribuicao']).groupby(
    ['ano_distribuicao', 'mes_distribuicao', 'dia_distribuicao'] + grouping_keys
).size().reset_index(name='Pendentes_dia')
pendentes_dia_df = pendentes_dia_df.rename(columns={'ano_distribuicao': 'ano', 'mes_distribuicao': 'mes', 'dia_distribuicao': 'dia'})


# --- 6. JUNÇÃO E LIMPEZA (ANUAL) ---
estatisticas_anuais = pd.merge(distribuidos_df, baixados_df, on=['ano'] + grouping_keys, how='outer')
estatisticas_anuais = pd.merge(estatisticas_anuais, pendentes_df, on=['ano'] + grouping_keys, how='outer')
estatisticas_anuais = estatisticas_anuais.fillna(0)
estatisticas_anuais[['Distribuídos', 'Baixados', 'Pendentes']] = estatisticas_anuais[['Distribuídos', 'Baixados', 'Pendentes']].astype(int)
estatisticas_anuais = estatisticas_anuais.dropna(subset=['ano'])
estatisticas_anuais['ano'] = estatisticas_anuais['ano'].astype(int)

# --- 7. JUNÇÃO E LIMPEZA (MENSAL) ---
merge_keys_mes = ['ano', 'mes'] + grouping_keys
estatisticas_mes = pd.merge(distribuidos_mes_df, baixados_mes_df, on=merge_keys_mes, how='outer')
estatisticas_mes = pd.merge(estatisticas_mes, pendentes_mes_df, on=merge_keys_mes, how='outer')
estatisticas_mes = estatisticas_mes.fillna(0)
estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']] = estatisticas_mes[['Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes']].astype(int)
estatisticas_mes = estatisticas_mes.dropna(subset=['ano', 'mes'])
estatisticas_mes[['ano', 'mes']] = estatisticas_mes[['ano', 'mes']].astype(int)

# --- 8. JUNÇÃO E LIMPEZA (SEMANAL) ---
merge_keys_semana = ['ano_semana', 'semana'] + grouping_keys
estatisticas_semana = pd.merge(distribuidos_semana_df, baixados_semana_df, on=merge_keys_semana, how='outer')
estatisticas_semana = pd.merge(estatisticas_semana, pendentes_semana_df, on=merge_keys_semana, how='outer')
estatisticas_semana = estatisticas_semana.fillna(0)
estatisticas_semana[['Distribuídos_semana', 'Baixados_semana', 'Pendentes_semana']] = estatisticas_semana[['Distribuídos_semana', 'Baixados_semana', 'Pendentes_semana']].astype(int)
estatisticas_semana = estatisticas_semana.dropna(subset=['ano_semana', 'semana'])
estatisticas_semana[['ano_semana', 'semana']] = estatisticas_semana[['ano_semana', 'semana']].astype(int)

# --- 9. JUNÇÃO E LIMPEZA (DIÁRIA) ---
merge_keys_dia = ['ano', 'mes', 'dia'] + grouping_keys
estatisticas_dia = pd.merge(distribuidos_dia_df, baixados_dia_df, on=merge_keys_dia, how='outer')
estatisticas_dia = pd.merge(estatisticas_dia, pendentes_dia_df, on=merge_keys_dia, how='outer')
estatisticas_dia = estatisticas_dia.fillna(0)
estatisticas_dia[['Distribuídos_dia', 'Baixados_dia', 'Pendentes_dia']] = estatisticas_dia[['Distribuídos_dia', 'Baixados_dia', 'Pendentes_dia']].astype(int)
estatisticas_dia = estatisticas_dia.dropna(subset=['ano', 'mes', 'dia'])
estatisticas_dia[['ano', 'mes', 'dia']] = estatisticas_dia[['ano', 'mes', 'dia']].astype(int)


# --- 10. CÁLCULO DAS TAXAS DE CONGESTIONAMENTO ---

# 10.1 Taxa ANUAL
soma_anual = estatisticas_anuais['Pendentes'] + estatisticas_anuais['Baixados']
estatisticas_anuais['Taxa de Congestionamento (%)'] = np.where(
    soma_anual > 0, (estatisticas_anuais['Pendentes'] / soma_anual) * 100, 0
).round(2)

# 10.2 Taxa MENSAL
soma_mensal = estatisticas_mes['Pendentes_mes'] + estatisticas_mes['Baixados_mes']
estatisticas_mes['Taxa de Congestionamento_mes (%)'] = np.where(
    soma_mensal > 0, (estatisticas_mes['Pendentes_mes'] / soma_mensal) * 100, 0
).round(2)

# 10.3 Taxa SEMANAL
soma_semanal = estatisticas_semana['Pendentes_semana'] + estatisticas_semana['Baixados_semana']
estatisticas_semana['Taxa de Congestionamento_semana (%)'] = np.where(
    soma_semanal > 0, (estatisticas_semana['Pendentes_semana'] / soma_semanal) * 100, 0
).round(2)

# 10.4 Taxa DIÁRIA
soma_diaria = estatisticas_dia['Pendentes_dia'] + estatisticas_dia['Baixados_dia']
estatisticas_dia['Taxa de Congestionamento_dia (%)'] = np.where(
    soma_diaria > 0, (estatisticas_dia['Pendentes_dia'] / soma_diaria) * 100, 0
).round(2)


# --- 11. ORDENAÇÃO E COLUNAS FINAIS ---

# 11.1 ANUAL
estatisticas_anuais = estatisticas_anuais.sort_values(by=['ano', 'comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True])
colunas_finais_anuais = [
    'ano', 'nome_area_acao', 'comarca', 'serventia',
    'Distribuídos', 'Baixados', 'Pendentes', 'Taxa de Congestionamento (%)'
]
estatisticas_anuais = estatisticas_anuais[colunas_finais_anuais]

# 11.2 MENSAL
estatisticas_mes = estatisticas_mes.sort_values(by=['ano', 'mes', 'comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True, True])
colunas_finais_mes = [
    'ano', 'mes', 'nome_area_acao', 'comarca', 'serventia',
    'Distribuídos_mes', 'Baixados_mes', 'Pendentes_mes', 'Taxa de Congestionamento_mes (%)'
]
estatisticas_mes = estatisticas_mes[colunas_finais_mes]

# 11.3 SEMANAL
estatisticas_semana = estatisticas_semana.sort_values(by=['ano_semana', 'semana', 'comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True, True])
colunas_finais_semana = [
    'ano_semana', 'semana', 'nome_area_acao', 'comarca', 'serventia',
    'Distribuídos_semana', 'Baixados_semana', 'Pendentes_semana', 'Taxa de Congestionamento_semana (%)'
]
estatisticas_semana = estatisticas_semana[colunas_finais_semana]

# 11.4 DIÁRIA
estatisticas_dia = estatisticas_dia.sort_values(by=['ano', 'mes', 'dia', 'comarca', 'serventia', 'nome_area_acao'], ascending=[False, True, True, True, True, True])
colunas_finais_dia = [
    'ano', 'mes', 'dia', 'nome_area_acao', 'comarca', 'serventia',
    'Distribuídos_dia', 'Baixados_dia', 'Pendentes_dia', 'Taxa de Congestionamento_dia (%)'
]
estatisticas_dia = estatisticas_dia[colunas_finais_dia]


Calculando estatísticas anuais...
Calculando estatísticas mensais...
Calculando estatísticas semanais...
Calculando estatísticas diárias...


In [ ]:
# GERAR CSV DE TAXA DE CONGESTIONAMENTO MENSAL ZERADA

# 1. Filtrar o DataFrame para linhas onde a taxa é zero
df_taxa_zero = estatisticas_mes[estatisticas_mes['Taxa de Congestionamento_mes (%)'] == 0]

# 2. Selecionar as colunas relevantes e remover duplicatas
df_comarcas_serventias_zero = df_taxa_zero[['comarca', 'serventia', 'ano', 'mes' ,'Taxa de Congestionamento_mes (%)']].drop_duplicates()

# 3. Ordenar os resultados
df_comarcas_serventias_zero = df_comarcas_serventias_zero.sort_values(by=['comarca', 'serventia'])

# 4. Salvar em um novo arquivo CSV
output_filename_taxa_zero_csv = 'comarcas_taxa_zero.csv'
df_comarcas_serventias_zero.to_csv(output_filename_taxa_zero_csv, index=False, encoding='utf-8-sig', sep=',')

In [21]:
# GERAR CSV DE TAXA DE CONGESTIONAMENTO MENSAL ZERADA COMARCAS E SERVENTIAS ÚNICAS

# 1. Filtrar o DataFrame para linhas onde a taxa é zero
df_taxa_zero = estatisticas_mes[estatisticas_mes['Taxa de Congestionamento_mes (%)'] == 0]

# 2. Selecionar as colunas relevantes e remover duplicatas
df_comarcas_serventias_zero = df_taxa_zero[['comarca', 'serventia','Taxa de Congestionamento_mes (%)']].drop_duplicates()

# 3. Ordenar os resultados
df_comarcas_serventias_zero = df_comarcas_serventias_zero.sort_values(by=['comarca', 'serventia'])

display(df_comarcas_serventias_zero.head(20))

# 4. Salvar em um novo arquivo CSV
output_filename_taxa_zero_csv = 'comarcas_taxa_zero.csv'
df_comarcas_serventias_zero.to_csv(output_filename_taxa_zero_csv, index=False, encoding='utf-8-sig', sep=',')

,comarca,serventia,Taxa de Congestionamento_mes (%)
54011,ABADIÂNIA,Vara Judicial,0.0
54017,ACREÚNA,"1ª Vara Judicial (Família e Sucessões, Infânci...",0.0
54023,ACREÚNA,"2ª Vara Judicial (Fazendas Públicas, Criminal,...",0.0
54032,ALEXÂNIA,Vara Judicial,0.0
55559,ALTO PARAÍSO DE GOIÁS,Vara Judicial,0.0
54047,ALVORADA DO NORTE,Vara Judicial,0.0
54053,ANICUNS,"1ª Vara Judicial (Família e Sucessões, Infânci...",0.0
54058,ANICUNS,"2ª Vara Judicial (Fazendas Públicas, Criminal,...",0.0
54060,ANICUNS,CENTRO JUDICIÁRIO DE SOLUÇÃO DE CONFLITOS E CI...,0.0
55582,ANÁPOLIS,1ª Vara Criminal,0.0


In [ ]:
# Gera o arquivo CSV com o dataframe completo por ano:
output_filename = 'taxa_congestionamento.csv'
estatisticas.to_csv(output_filename, index=False, encoding='utf-8-sig', sep=',')

In [9]:
# Gera o arquivo CSV com o dataframe completo por mês
output_filename_mes = 'taxa_cong_mensal.csv'
estatisticas_mes.to_csv(output_filename_mes, index=False, encoding='utf-8-sig', sep=',')

In [10]:
# Gera o arquivo JSON com o dataframe completo
output_filename_json = 'tx_cong.json'
estatisticas.to_json(output_filename_json, orient='records', indent=4, force_ascii=False)

In [10]:
# Gera o arquivo JSON com o dataframe completo por mês
output_filename_json_mes = 'tx_cong_mensal.json'
estatisticas_mes.to_json(output_filename_json_mes, orient='records', indent=4, force_ascii=False)

In [7]:
# --- VISUALIZAÇÃO E EXPORTAÇÃO DOS RESULTADOS ---

"""# ANUAL
print("\nTabelão Geral de Estatísticas Anuais (10 primeiras linhas):")
print(estatisticas_anuais.head(10))
estatisticas_anuais.to_csv('estatisticas_anuais.csv', index=False, encoding='utf-8-sig', sep=',')
estatisticas_anuais.to_json('estatisticas_anuais.json', orient='records', indent=4, force_ascii=False)
print("Arquivos anuais salvos com sucesso.")

# MENSAL
print("\nTabelão Geral de Estatísticas Mensais (10 primeiras linhas):")
print(estatisticas_mes.head(10))
estatisticas_mes.to_csv('estatisticas_mensais.csv', index=False, encoding='utf-8-sig', sep=',')
estatisticas_mes.to_json('estatisticas_mensais.json', orient='records', indent=4, force_ascii=False)
print("Arquivos mensais salvos com sucesso.")

# SEMANAL
print("\nTabelão Geral de Estatísticas Semanais (10 primeiras linhas):")
print(estatisticas_semana.head(10))
estatisticas_semana.to_csv('estatisticas_semanais.csv', index=False, encoding='utf-8-sig', sep=',')
estatisticas_semana.to_json('estatisticas_semanais.json', orient='records', indent=4, force_ascii=False)
print("Arquivos semanais salvos com sucesso.")
"""
# DIÁRIO
print("\nTabelão Geral de Estatísticas Diárias (10 primeiras linhas):")
print(estatisticas_dia.head(10))
estatisticas_dia.to_csv('estatisticas_diarias.csv', index=False, encoding='utf-8-sig', sep=',')
estatisticas_dia.to_json('estatisticas_diarias.json', orient='records', indent=4, force_ascii=False)
print("Arquivos diários salvos com sucesso.")


# --- NOVA SEÇÃO: EXPORTAÇÃO COMBINADA ---
"""print("\nGerando arquivos combinados (anuais, mensais, semanais, diários)...")

# 1. Concatenar todos os DataFrames de estatísticas
#    'ignore_index=True' é essencial para criar um novo índice
#    'sort=False' previne o warning de ordenação de colunas
estatisticas_combinadas = pd.concat(
    [estatisticas_anuais, estatisticas_mes, estatisticas_semana, estatisticas_dia],
    ignore_index=True,
    sort=False
)

# 2. Ordenar o DataFrame combinado para melhor visualização
#    Ordena primeiro pela localidade, depois pelos componentes de data
estatisticas_combinadas = estatisticas_combinadas.sort_values(
    by=['comarca', 'serventia', 'nome_area_acao', 'ano', 'mes', 'dia', 'ano_semana', 'semana'],
    na_position='first' # Agrupa por localidade primeiro
)

# 3. Visualizar e Exportar o DataFrame combinado
print("\nTabelão Geral de Estatísticas Combinadas (10 primeiras linhas):")
print(estatisticas_combinadas.head(10))

estatisticas_combinadas.to_csv('estatisticas_combinadas.csv', index=False, encoding='utf-8-sig', sep=',')
estatisticas_combinadas.to_json('estatisticas_combinadas.json', orient='records', indent=4, force_ascii=False)
print("Arquivos combinados salvos com sucesso.")"""



Tabelão Geral de Estatísticas Diárias (10 primeiras linhas):
         ano  mes  dia             nome_area_acao                comarca  \
745678  2025    1    1                   criminal                ACREÚNA   
745679  2025    1    1  juizado especial criminal               ALEXÂNIA   
745680  2025    1    1                   criminal  ALTO PARAÍSO DE GOIÁS   
745681  2025    1    1                      civel      ALVORADA DO NORTE   
745682  2025    1    1  juizado especial criminal      ALVORADA DO NORTE   
745683  2025    1    1     juizado especial civel               ANÁPOLIS   
745684  2025    1    1               upj criminal               ANÁPOLIS   
745685  2025    1    1     juizado especial civel               ANÁPOLIS   
745686  2025    1    1                   criminal               ANÁPOLIS   
745687  2025    1    1     juizado especial civel               ANÁPOLIS   

                                                serventia  Distribuídos_dia  \
745678  2ª Vara Judici

'print("\nGerando arquivos combinados (anuais, mensais, semanais, diários)...")\n\n# 1. Concatenar todos os DataFrames de estatísticas\n#    \'ignore_index=True\' é essencial para criar um novo índice\n#    \'sort=False\' previne o warning de ordenação de colunas\nestatisticas_combinadas = pd.concat(\n    [estatisticas_anuais, estatisticas_mes, estatisticas_semana, estatisticas_dia],\n    ignore_index=True,\n    sort=False\n)\n\n# 2. Ordenar o DataFrame combinado para melhor visualização\n#    Ordena primeiro pela localidade, depois pelos componentes de data\nestatisticas_combinadas = estatisticas_combinadas.sort_values(\n    by=[\'comarca\', \'serventia\', \'nome_area_acao\', \'ano\', \'mes\', \'dia\', \'ano_semana\', \'semana\'],\n    na_position=\'first\' # Agrupa por localidade primeiro\n)\n\n# 3. Visualizar e Exportar o DataFrame combinado\nprint("\nTabelão Geral de Estatísticas Combinadas (10 primeiras linhas):")\nprint(estatisticas_combinadas.head(10))\n\nestatisticas_combinadas